
# Django - Customise Your SQL



## Josh Smeaton
<hr>
- jarshwah on #django / Twitter / Github
- Software Engineer at Engage/LivePerson
- SQL Nerd
- Django Core Developer
- Primary author of Expressions


## Recent Developments
<hr>
- Lookups (1.7)

```python
Product.objects.filter(tagarray__contains=['large', 'jumper'])
```

- Transforms (1.7)

```python
Product.objects.filter(name__lower__contains='jumper')
```

- Expressions (1.8)

```python
Sale.objects.annotate(
    sale_month=TruncMonth('sale_date')
).values('sale_month').annotate(sales_per_month=Count('id'))
```


## Sales Reporting

In [1]:
from IPython.display import display
from shop.functions import table
qs = Sale.objects.values('product__category').annotate(
    sales=Count('id'),
    revenue=Sum('sale_price')
)
display(table(qs))

,product__category,revenue,sales
0,Women,39359.00,332
1,Men,41281.00,340
2,Kids,10245.00,328


In [2]:
from IPython.display import display
from shop.functions import sql, dataframe
df = dataframe(Product.objects.values().order_by('cost_price'))
display(df)

,category,cost_price,id,name
0,Kids,10.00,62,Gloves
1,Kids,15.00,61,Gum Boots
2,Women,18.50,60,Scarf
3,Men,19.00,57,Beanie
4,Kids,35.00,63,Rain Jacket
5,Women,90.00,58,Boots
6,Men,120.00,55,Leather Shoes
7,Men,190.00,56,Winter Coat
8,Women,210.00,59,Long Overcoat


In [3]:
from shop.functions import sql, dataframe
print(sql(Product.objects.all()))

SELECT "shop_product"."id",
       "shop_product"."name",
       "shop_product"."category",
       "shop_product"."cost_price"
FROM "shop_product"
